# 1. Setup

Import the libraries, set up the GLOBAL variables, and run the google authentications 

In [ ]:
# Import libraries
import os
import rasterio
import glob
import json
import tensorflow as tf
import folium
import ee
from pprint import pprint
from google.oauth2 import service_account
from google.cloud import storage

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Specify the key for accessing the google cloud project
KEY_PATH = '/home/climber/.keys/ee-maptheforests-access-key.json'

# Our bucket
BUCKET_NAME = 'naip_structure'

# Set the directories where we will want to grab TFrecords from on gcloud
GCLOUD_NAIP_DIR  = 'naip_Malheur_TFrecords/'
#GCLOUD_NAIP_DIR  = 'naip_Malheur_final_TFrecords/2011/''

# Where to put them on Treenet
TREENET_NAIP_DIR = '/scratch/NAIP_data/malheur/2011/'

# Name of the output raster name
TIF_NAME = 'Malheur_bbox_predicted_canCov_v1.tif'

# Assets folder in GEE
GEE_USER_FOLDER = 'users/forestMapper'

# Input bands to the model
BANDS = ['B','G','R','N']

# The shape of patches expected by the model
KERNEL_SHAPE = [30, 30]

#Speicfy where the model file is on the different platforms
MODEL_ON_GCLOUD = 'gs://' + BUCKET_NAME + '/CNN_models/CNN_canCov_v1'
MODEL_ON_TREENET = '/scratch/CNN/models/canCov_model_2011_v1/'

#Authenticate and intialize earth engine
ee.Authenticate()
ee.Initialize()

# The bounding box of the area we'd like to map over
MAP_REGION = ee.Geometry.Polygon(
        [[[-119.37239004502233,44.48079613290612],
          [-118.57725454697545,44.48079613290612],
          [-118.57725454697545,44.81785572318615],
          [-119.37239004502233,44.81785572318615]]], None, False)

# MAP_REGION = ee.Geometry.Polygon(
#         [[[-120.5141935519472,43.58307281277678],
#           [-117.88022138397845,43.58307281277678],
#           [-117.88022138397845,44.84291584106584],
#           [-120.5141935519472,44.84291584106584]]], None, False)


# print(f'Available hardware: {tf.config.list_physical_devices()}')
# print(f'Tensorflow version {tf.__version__}')
# print(f'folium version {folium.__version__}')

# 2. Show imagery of the requested bbox

In [ ]:
# The NAIP image collection
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# The image input data is a cloud-masked median composite.
image = naip.filterDate('2011-01-01', '2011-12-31').median().clip(MAP_REGION)

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255})
map = folium.Map(location=[44.62157, -118.98257])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

map

# 3. Export imagery GEE --> Gcloud

In [ ]:
def doExport(out_image_base, region):

  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = TF_FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 1,
    fileFormat = 'TFRecord',
    maxPixels = 1e13,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
#Run the do export function (somehow I lost this but it's only one line of code)

# 4. Pull imagery Gcloud --> Treenet

In [ ]:
# Change directory to be where we want to write the data
os.chdir(TREENET_NAIP_DIR)

# Credentials for the google cloud connection
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)

# Open a gcloud client session using the "credentials" object
client = storage.Client(credentials=credentials)

# Get the bucket and the blob object
bucket = client.bucket(BUCKET_NAME)

# Get the blobs from our bucket (list of files within the gcloud_naip_dir directory)
blobs = bucket.list_blobs(prefix = GCLOUD_NAIP_DIR)

# Loop through the blobs
for blob in blobs:
    
    # get the filename without the leading directories
    file_name = blob.name.split('/')[-1]  
    
    # Skip the blob that is just the directory name
    if len(file_name) < 5: 
        continue
        
    # Grab the file    
    #blob.download_to_filename(file_name)
    
    # Print that the file is done
    print(file_name) 

# Run away!
print('Run away!')